In [ ]:
import os

from ipywidgets import Text, Textarea, jslink, link, HBox, VBox, HTML
from ipytree import Tree, Node
from ipy_table import *

from src.pipeline.repository import Repository

In [ ]:
data_repo_path = os.path.join('..', 'data', 'repository')
repo = Repository(data_repo_path)

In [ ]:
id_text = Text()
task_function = Textarea()
option_task = HTML()

In [ ]:
def get_id_from_node(n):
    return n.name.split('/')[1].strip()

def get_task(id):
    ds = repo.metadata['datasets'][id]
    task_name = ''
    if 'task_config' in ds:
        task_name = str(ds['task_config']['pipeline']['data']['tasks']['task_func']['_target_'])
        task_name = task_name.split('.')[-1]
    return task_name

def make_table_from_dict(d: dict):
    # lists
    temp = []
    dictList = []
    # convert the dictionary to a list
    for key, value in d.items():     
        temp = [key,value]
        dictList.append(temp)
    make_table(dictList)
    return render()._repr_html_()

def handle_click(event):
    ds_id = get_id_from_node(event['owner'])
    ds = repo.metadata['datasets'][ds_id]
    task_name = get_task(ds_id)
    
    id_text.value = ds_id
    task_function.value = ''
    option_task.value = ''
    if 'task_config' in ds:
        task_function.value = get_task(ds_id)
        option_task.value = make_table_from_dict(ds['task_config']['pipeline']['data']['tasks']['task_func'])
    
       
def create_tree_from_repo(repo: Repository):
    tree = Tree()
    list_node = []
    datatsets = repo.metadata['datasets']
    for ds_id in datatsets.keys():
        task_name = get_task(ds_id)
        n = Node(task_name + " / " + ds_id)      
        n.observe(handle_click, 'selected')
        list_node.append(n)
    
    for n in list_node:
        ds_id = get_id_from_node(n)
        if 'task_config' not in datatsets[ds_id]:
            tree.add_node(n)
        else:
            src_id = datatsets[ds_id]['task_config']['pipeline']['data']['dataset']['source']
            parent_node = [n for n in list_node if src_id in n.name][0]
            parent_node.add_node(n)            
    return tree



In [ ]:
repo_tree = create_tree_from_repo(repo)

In [ ]:
HBox(children=[ repo_tree, VBox(children = [id_text, task_function, option_task])])